In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import root_mean_squared_error,r2_score

In [136]:
train_df = pd.read_csv('data/Train.csv')

In [88]:
data_train = pd.read_csv('Train_modified.csv',parse_dates=['DATOP','STA','STD'])
data_train.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,AC,target,ATA,...,month_DATOP,year_DATOP,weekdays_DATOP,flight_duration_hrs,flight_duration_category,fl_op,ac_type,flight_length_cat,flight_duration_bin,time_of_day_cat
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,TU 32AIMN,260.0,True,...,1,2016,6,2.416667,long,1.0,1.0,medium,120-179,morning
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,TU 31BIMO,20.0,True,...,1,2016,2,1.833333,medium,1.0,2.0,medium,60-119,afternoon
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,TU 32AIMN,0.0,True,...,1,2016,5,2.583333,long,1.0,1.0,medium,120-179,night
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,TU 736IOK,0.0,True,...,1,2016,6,2.833333,long,1.0,3.0,medium,120-179,midday
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,TU 320IMU,22.0,True,...,1,2016,6,1.333333,medium,1.0,1.0,short,60-119,midday


In [130]:
df = data_train.copy()

In [131]:
df['STATUS'] = train_df.STATUS

In [132]:
df.drop(['ID','DATOP','STD','STA','flight_duration_bin','flight_length_cat','ATA','SCH'],axis=1,inplace=True)

In [133]:
df.head()

,FLTID,DEPSTN,ARRSTN,AC,target,month_DATOP,year_DATOP,weekdays_DATOP,flight_duration_hrs,flight_duration_category,fl_op,ac_type,time_of_day_cat,STATUS
0,TU 0712,CMN,TUN,TU 32AIMN,260.0,1,2016,6,2.416667,long,1.0,1.0,morning,ATA
1,TU 0757,MXP,TUN,TU 31BIMO,20.0,1,2016,2,1.833333,medium,1.0,2.0,afternoon,ATA
2,TU 0214,TUN,IST,TU 32AIMN,0.0,1,2016,5,2.583333,long,1.0,1.0,night,ATA
3,TU 0480,DJE,NTE,TU 736IOK,0.0,1,2016,6,2.833333,long,1.0,3.0,midday,ATA
4,TU 0338,TUN,ALG,TU 320IMU,22.0,1,2016,6,1.333333,medium,1.0,1.0,midday,ATA


In [134]:
df.STATUS = df.STATUS.astype('category')
df.month_DATOP = df.month_DATOP.astype('category')
df.year_DATOP = df.year_DATOP.astype('category')
df.weekdays_DATOP = df.weekdays_DATOP.astype('category')
df.fl_op = df.fl_op.astype('category')
df.ac_type = df.ac_type.astype('category')
df.flight_duration_category = df.flight_duration_category.astype('category')
df.time_of_day_cat = df.time_of_day_cat.astype('category')
df.ARRSTN = df.ARRSTN.astype('category')
df.DEPSTN = df.DEPSTN.astype('category')
df.AC = df.AC.astype('category')
df.FLTID = df.FLTID.astype('category')



In [135]:
cat_features = df.select_dtypes(include=['category']).columns.to_list()
cat_features

['FLTID',
 'DEPSTN',
 'ARRSTN',
 'AC',
 'month_DATOP',
 'year_DATOP',
 'weekdays_DATOP',
 'flight_duration_category',
 'fl_op',
 'ac_type',
 'time_of_day_cat',
 'STATUS']

In [ ]:
df

In [94]:
num_feature = df.select_dtypes(exclude=['category','datetime64']).columns
num_feature = num_feature.drop('target').to_list()
num_feature

['flight_duration_hrs']

In [100]:
df.columns

Index(['FLTID', 'DEPSTN', 'ARRSTN', 'AC', 'target', 'ATA', 'SCH',
       'month_DATOP', 'year_DATOP', 'weekdays_DATOP', 'flight_duration_hrs',
       'flight_duration_category', 'fl_op', 'ac_type', 'time_of_day_cat'],
      dtype='object')

In [113]:
for  cat in cat_features:
    print(f'{cat} has unique values ----- {df[cat].nunique()}')

FLTID has unique values ----- 1861
DEPSTN has unique values ----- 132
ARRSTN has unique values ----- 128
AC has unique values ----- 68
ATA has unique values ----- 2
SCH has unique values ----- 2
month_DATOP has unique values ----- 12
year_DATOP has unique values ----- 3
weekdays_DATOP has unique values ----- 7
flight_duration_category has unique values ----- 3
fl_op has unique values ----- 4
ac_type has unique values ----- 8
time_of_day_cat has unique values ----- 5


In [101]:
y = df['target']
X = df.drop('target',axis=1)

In [ ]:
train_X , test_x ,train_y,test_y = train_test_split(X,y,test_size=0.3,random_state=42)

In [103]:
print('Training Set :',train_X.shape , train_y.shape)
print('Training Set :',test_x.shape , test_y.shape)

Training Set : (75483, 14) (75483,)
Training Set : (32350, 14) (32350,)


In [ ]:
model_rf = RandomForestRegressor()
model_rf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [ ]:
grid_param_rf ={'n_estimators': [50,100,150,200],
                'criterion' :['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
                'max_depth':[10,20,30,40,50],
                'max_features':['sqrt','log2',None],
                'n_jobs': -1
}
rand_fr = GridSearchCV(model_rf,grid_param_rf,scoring='root_mean_squared_error',n_jobs=-1)
rand_fr.fit(train_X,train_y)
rand_fr.best_params_